In [1]:
from helper import create_batches
import tensorflow as tf
import numpy as np
import shelve
import joblib
import time
import math
import random

#Cuz the file is inside 'code' directory
mount_point = "../shelved_data/"

with shelve.open(mount_point+'IAM_Data') as shelf:
    vocabulary = shelf['chars']
    list_of_images = shelf['list_of_images']
    image_labels = shelf['image_labels']
    
image_arrays = joblib.load(mount_point+'image_arrays')

# #List_images ko sort karo
# list_of_images.sort()

#Convert vocabulary to list
vocabulary = list(vocabulary)
#Sort so as to have the same ordering every time..
vocabulary.sort()
vocabulary.append("<Blank>")

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
#Model parameters
img_height = 104
img_width = 688
vocab_size = len(vocabulary)

#Common Hyper Parameters
epochs = 500
#Should be proportional to the number of Images
#should be a multiple of 13000
batch_size = 200 

#LSTM Params
rnn_hidden_units = 200
rnn_layers = 5

conv_out_height, conv_out_width = (int(math.ceil(img_height/(2**3 * 3))),int(math.ceil(img_width/(2**3 * 3))))

#Number of time_steps to unroll for..
seq_len = conv_out_height * conv_out_width

## Save my MoDel

In [3]:
#TODO: Separate Training, testing and validation images into different lists..
random.shuffle(list_of_images)
train_size = 13000
test_size = 1000
valid_size = 94

training_list = list_of_images[:train_size]
testing_list = list_of_images[train_size:train_size+test_size]
validation_list = list_of_images[train_size+test_size:]

In [4]:
test_batches_x,test_batches_y = create_batches(len(testing_list),testing_list,image_arrays,image_labels,vocabulary)
valid_batches_x,valid_batches_y = create_batches(len(validation_list),validation_list,image_arrays,image_labels,vocabulary)

## Predict using Model

In [5]:
resume_epoch = 390

In [ ]:
with tf.Session() as sess:
    # Load the graph
    saver = tf.train.import_meta_graph('../model/200_5_Lines_RNN_'+str(resume_epoch)+'.meta')

    # Restore the weights and biases
    saver.restore(sess, '../model/200_5_Lines_RNN_'+str(resume_epoch))

    inputs = sess.graph.get_tensor_by_name('Placeholder:0')
    
    target_indices = sess.graph.get_tensor_by_name('targets/indices:0')
    target_values = sess.graph.get_tensor_by_name('targets/values:0')
    target_shape = sess.graph.get_tensor_by_name('targets/shape:0')

    time_steps = sess.graph.get_tensor_by_name('Placeholder_1:0')
    dropout_lstm = sess.graph.get_tensor_by_name('Placeholder_2:0')
    dropout_fc = sess.graph.get_tensor_by_name('Placeholder_3:0')
    
        
    train = sess.graph.get_operation_by_name('RMSProp')
    cost = sess.graph.get_tensor_by_name('Mean:0')
    label_error_rate = sess.graph.get_tensor_by_name('Mean_1:0')
    

    #checkpoint flag
    checkpoint = False
    
    timer = 0
    
    for e in range(resume_epoch,epochs): 

        start_time = time.time()

        #Shuffle the training_list
        random.shuffle(training_list)
        
        #Create new training_batches..
        train_batches_x,train_batches_y = create_batches(batch_size,training_list,image_arrays,image_labels,vocabulary)
    
        #Checkpoint every 2 epochs
        if (e%1)==0:
            checkpoint = True
        
        total_cost = 0.0
        total_ler = 0.0
        #Iterate through all images in a single epoch...
        for b in range(len(train_batches_x)):
            
            feed_train = {
                    inputs:train_batches_x[b].transpose([2,0,1]),
                    target_indices:train_batches_y[b][0],target_values:train_batches_y[b][1],target_shape:train_batches_y[b][2],
                    time_steps:np.array([seq_len]*batch_size),
                    dropout_fc:np.array(0.9),dropout_lstm:np.array(0.90)
                   }
        
            _,cost_val,train_ler_val = sess.run([train,cost,label_error_rate],feed_dict=feed_train)

            total_cost+=cost_val
            total_ler+=train_ler_val

#         if checkpoint:
#             last_cost,train_ler = sess.run([cost,label_error_rate],feed_dict=feed_train)

        avg_cost = total_cost/len(train_batches_x) 
        avg_ler = total_ler/len(train_batches_x)
        
        #After iterating through all batches..
        valid_batch_size = len(validation_list)

      
        feed_valid = {
            inputs:valid_batches_x[0].transpose([2,0,1]),
            target_indices:valid_batches_y[0][0],target_values:valid_batches_y[0][1],target_shape:valid_batches_y[0][2],
            time_steps:np.array([seq_len]*valid_batch_size),
            dropout_fc:np.array(1.0),dropout_lstm:np.array(1.0)
           }
            
        #After iterating through all batches..
        test_batch_size = len(testing_list)
      
        feed_test = {
            inputs:test_batches_x[0].transpose([2,0,1]),
            target_indices:test_batches_y[0][0],target_values:test_batches_y[0][1],target_shape:test_batches_y[0][2],
            time_steps:np.array([seq_len]*test_batch_size),
            dropout_fc:np.array(1.0),dropout_lstm:np.array(1.0)
           }
            

       #Evaluate the model, and store every 5 epochs...
        if checkpoint:
            #Accuracy on valid_data
            test_ler = sess.run(label_error_rate,feed_dict=feed_valid)                

            end_time = time.time()       
            time_taken = end_time - start_time
            timer += time_taken
     
            print("{},{:.6f},{:.2f},{:.2f},{}\n".format(e,avg_cost,avg_ler,test_ler,timer))

            with open('progress.csv','a') as f:
                f.write("{},{:.6f},{:.2f},{:.2f},{}\n".format(e,avg_cost,avg_ler,test_ler,timer))

            #Save the model
            saver.save(sess,'../model/200_5_Lines_RNN_'+str(e))

            #Reset the checkpoint-flag and timer
            checkpoint = False
            timer = 0
        
        else:
            end_time = time.time()       
            time_taken = end_time - start_time
            timer += time_taken

INFO:tensorflow:Restoring parameters from ../model/200_5_Lines_RNN_320
320,117.536722,0.66,0.71,97.09253597259521

INFO:tensorflow:../model/200_5_Lines_RNN_320 is not in all_model_checkpoint_paths. Manually adding it.
321,111.007619,0.66,0.67,91.89549040794373

INFO:tensorflow:../model/200_5_Lines_RNN_321 is not in all_model_checkpoint_paths. Manually adding it.
322,109.190777,0.65,0.72,92.2608163356781

INFO:tensorflow:../model/200_5_Lines_RNN_322 is not in all_model_checkpoint_paths. Manually adding it.
323,107.787246,0.65,0.69,91.666024684906

INFO:tensorflow:../model/200_5_Lines_RNN_323 is not in all_model_checkpoint_paths. Manually adding it.
324,106.981319,0.65,0.73,91.90571308135986

INFO:tensorflow:../model/200_5_Lines_RNN_324 is not in all_model_checkpoint_paths. Manually adding it.
325,106.315491,0.64,0.68,91.3876326084137

INFO:tensorflow:../model/200_5_Lines_RNN_325 is not in all_model_checkpoint_paths. Manually adding it.
326,105.597002,0.64,0.72,93.65916419029236

INFO:te

376,96.650173,0.60,0.68,88.38164591789246

INFO:tensorflow:../model/200_5_Lines_RNN_376 is not in all_model_checkpoint_paths. Manually adding it.
377,96.526900,0.60,0.70,88.65398907661438

INFO:tensorflow:../model/200_5_Lines_RNN_377 is not in all_model_checkpoint_paths. Manually adding it.
378,96.484034,0.60,0.69,88.3535304069519

INFO:tensorflow:../model/200_5_Lines_RNN_378 is not in all_model_checkpoint_paths. Manually adding it.
379,96.413939,0.60,0.68,89.0223228931427

INFO:tensorflow:../model/200_5_Lines_RNN_379 is not in all_model_checkpoint_paths. Manually adding it.
